In [23]:
import h5py
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

In [28]:
fname_perturbed = 'E:/Dima/PhD/Papers/data/states/states_model_60_4_80_500_r2.h5py'
fname_original = 'E:\Dima\PhD\Papers\data\states\states_model_60_4_80_500.h5py'
tokens_file = 'E:/Dima/PhD/Papers/data/states/tokens_index_model_60_4_80_500.csv'

In [29]:
tt = h5py.File(open(fname_perturbed, 'rb'), 'r')
f = h5py.File(fname_original, 'r')
df_tokens = pd.read_csv(tokens_file)
df_tokens.head()

,id,token
0,0,preface supposing that truth is a womanwhat ...
1,1,reface supposing that truth is a womanwhat t...
2,2,eface supposing that truth is a womanwhat th...
3,3,face supposing that truth is a womanwhat the...
4,4,ace supposing that truth is a womanwhat then...


In [30]:
np.argmax(tt['y_0'][()])

38

In [31]:
np.argmax(f['y_' + str(0)][()])

38

In [32]:
k = 0

p3_satisfies = []

for elem_id, row in df_tokens.iterrows():
    elem = row[1] 
    p_perturbed = np.argmax(tt['y_' + str(elem_id)][()]) # position of the element that has the highest probability
    p_original = np.argmax(f['y_' + str(elem_id)][()])
    
    if (p_perturbed == p_original):
        p3_satisfies.append(0)
    else:
        p3_satisfies.append(1)
    
    k = k + 1
    if k % 1000 == 0:
        print("elem = {}. k = {}.".format(elem, k))

elem = ces as the dogmatists have hitherto reared perhaps. k = 1000.
elem = edanta doctrine in asia and platonism in europe le. k = 2000.
elem = e against the ecclesiastical oppression of millenn. k = 3000.
elem =  knows the goal to aim at  sils maria upper engadi. k = 4000.
elem = ented itself before usor was it we who presented o. k = 5000.
elem = e intransitory in the concealed god in the thingin. k = 6000.
elem = es as it were to borrow an expression current amon. k = 7000.
elem = ers and having read between their lines long enoug. k = 8000.
elem = valuations special kinds of _niaiserie_ such as ma. k = 9000.
elem =  manner and a philosophy which ventures to do so h. k = 10000.
elem = erally astute defenders also of their prejudices w. k = 11000.
elem = sonal timidity and vulnerability does this masquer. k = 12000.
elem = l practiced philosophy at one time or another and . k = 13000.
elem = r that in the philosopher on the contrary there is. k = 14000.
elem =  garden at athens and 

In [33]:
df_p3 = pd.DataFrame({'p3':p3_satisfies})

In [34]:
df_p3.p3.value_counts()

0    354767
1    225974
Name: p3, dtype: int64

In [35]:
225974/df_p3.shape[0]

0.3891132191458843

# SMC

In [36]:
def get_nu_rho(n_satisfies, n_sampled_so_far, alpha, beta):
    ro = (n_satisfies + alpha) / (n_sampled_so_far + alpha + beta)
    nu = np.sqrt(((alpha + n_satisfies) * (n_sampled_so_far - n_satisfies + beta)) / (
            pow((alpha + n_sampled_so_far + beta), 2) * (alpha + n_sampled_so_far + beta + 1)))
    return ro, nu

In [37]:
# calculate time
elements_num = df_tokens.shape[0]
tmp = list(range(0, elements_num))
tokens_indx_rand = tmp.copy()
random.seed(42)
random.shuffle(tokens_indx_rand)
alpha = 1
beta = 1

n_satisfies_p3_list = []
n_sampled_so_far_list = []

smc_ro_estimates_p3 = []
smc_nu_estimates_p3 = []

n_satisfies_p3 = 0
n_sampled_so_far = 0

k = 0

for i in tokens_indx_rand:
    # TODO: vectorize this function using apply()
    # should be cumulative group by operation

    if p3_satisfies[i] == 1:
        n_satisfies_p3 = n_satisfies_p3 + 1

    n_sampled_so_far = n_sampled_so_far + 1
    rho_p3, nu_p3 = get_nu_rho(n_satisfies_p3, n_sampled_so_far, alpha, beta)

    n_satisfies_p3_list.append(n_satisfies_p3)
    n_sampled_so_far_list.append(n_sampled_so_far)

    smc_ro_estimates_p3.append(rho_p3)
    smc_nu_estimates_p3.append(nu_p3)
#     if k % 1000 == 0:
#         print(" k = {}.".format(k))
#     k = k + 1
    
_df_p3_smc = pd.DataFrame({'token_id': tokens_indx_rand,
                          'satisfies_p3': n_satisfies_p3_list,
                          'sampled': n_sampled_so_far_list,
                          'rho_p3': smc_ro_estimates_p3,
                          'nu_p3': smc_nu_estimates_p3
                          })

In [38]:
model_base_name = 'model_60_4_80_500'
verify_results_folder='E:/Dima/PhD/Papers/data/verify_results/'

In [39]:
_df_p3_smc.to_csv(verify_results_folder + "/" + model_base_name + "_p3_smc.csv", index=False)